<a href="https://colab.research.google.com/github/ShravaniPathak/Superstore-Profit-Prediction/blob/main/Superstore_Dataset_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'superstore-dataset-final:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1940216%2F3196104%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240929%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240929T184953Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7a6c86d77c419d122363e4210d2e5636d90d03bbeb0e14615d34da230fab8e47061c37fd7b660d08bb46c26c1247ba70af24483cee626beb17099ce0e656f12763d69b0c336fa4892ca481d6a6c07b025aac8af8d43d1997fb0b3e45e856a4a9ca4b2371e27cb00d90f7c85499ef44d4e675a8a5f604d4160a5a64e3b90adbac83e66ae72209d1fda9b8e55e30d669e420b2ea3c18243b72abf1062f2380f6e9d6f5e041d58902495af50650ea24e444f4d1302b9888f3338ba4ab1c83157d337fcb214569f738adf2dfdace1df4bd0ff4a62d8fe0c3f1f02d04ef55d0eb1ee0e3cd83a8cb17ad6a53b4314e70d85f253363b174f70e5063fc809f08abc88c4c'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


In [ ]:
df = pd.read_csv("/kaggle/input/superstore-dataset-final/Sample - Superstore.csv", encoding='windows-1252')


In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
for i in df.columns:
    j=df[i].nunique()
    print(f"{i} : {j}")

In [ ]:
Num_df=df.select_dtypes(include=['number'])
Num_df


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

model=LinearRegression()


In [ ]:
from sklearn.preprocessing import StandardScaler
# Take Sales as independent variable and Profit as dependent variable
X = df["Sales"]
y = df["Profit"]

# Reshape X to be 2D for training/testing
X = X.values.reshape(-1, 1)

# Splitting the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Standardizing the data
scaler = StandardScaler()
X_train_final = scaler.fit_transform(X_train)
X_test_final = scaler.transform(X_test)


In [ ]:
X_train.shape,y_train.shape,X_val.shape,y_test.shape


In [ ]:
model.fit(X_train_final,y_train)

In [ ]:
prediction=model.predict(X_train_final)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

rmse_train_lin=(mean_squared_error(y_train,prediction))**0.5
amse_train_lin=mean_absolute_error(y_train,prediction)


print(f'RMSE  Train By Linear -----> {rmse_train_lin:0.2f} ')
print(f'AMSE  Train By Linear -----> {amse_train_lin:0.2f} ')


In [ ]:
test_prediction=model.predict(X_test)

In [ ]:
rmse_test_lin=(mean_squared_error(y_test,test_prediction))**0.5
amse_test_lin=mean_absolute_error(y_test,test_prediction)


print(f'RMSE  test By Linear -----> {rmse_test_lin:0.2f} ')
print(f'AMSE  test By Linear -----> {amse_test_lin:0.2f} ')
